In [ ]:
!pip install tensorflow

In [ ]:
!pip install -U scikit-learn

In [1]:
# Importando as bibliotecas necessárias
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from PIL import Image

# Configurando o matplotlib
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [10, 5]

In [3]:
# Download
# https://www.dropbox.com/s/8o4ysiccspp46ov/skin-cancer-mnist-ham10000.zip?dl=1

# Load the metadata
metadata = pd.read_csv('data/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

# Print the first 5 rows of the metadata
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
# Preparação dos dados

# Carregar as imagens e os rótulos
images = []
labels = []

# Caminho para as imagens
image_path = 'data/skin-cancer-mnist-ham10000/HAM10000_images_part_1/'

# Loop através de todas as imagens no diretório
for img in os.listdir(image_path):
    # Carregar a imagem
    image = Image.open(image_path + img)
    # Redimensionar a imagem para 32x32 pixels
    image = image.resize((32, 32))
    # Converter a imagem para array numpy
    image = np.array(image)
    # Adicionar a imagem à lista de imagens
    images.append(image)

    # Adicionar o rótulo à lista de rótulos
    labels.append(metadata[metadata['image_id'] == img.split('.')[0]]['dx'].values[0])

# Converter listas em arrays numpy
images = np.array(images)
labels = np.array(labels)

# Print the shape of the images and labels
print('Images shape:', images.shape)
print('Labels shape:', labels.shape)

In [ ]:
# Codificar os rótulos
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels)

# Dividir os dados em conjuntos de treinamento, validação e teste
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, stratify=labels, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Print the shape of the train, validation and test sets
print('Train set shape:', x_train.shape, y_train.shape)
print('Validation set shape:', x_val.shape, y_val.shape)
print('Test set shape:', x_test.shape, y_test.shape)

NameError: name 'labels' is not defined

In [ ]:
# Verificar se os rótulos foram corretamente extraídos do dataset
print('Labels:', labels)

NameError: name 'labels' is not defined

In [ ]:
# Definindo a arquitetura da rede
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

# Print the model summary
model.summary()

In [ ]:
# Compilando o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinando o modelo
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

In [ ]:
# Plotando a acurácia de treinamento e validação ao longo das épocas
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Avaliando o modelo no conjunto de teste
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)